In [68]:
import mysql.connector
from mysql.connector import errorcode

In [103]:
config = {
    'host': '127.0.0.1',  # or your MySQL server host
    # Add 'user' and 'password' fields if your MySQL server requires them
}

In [119]:
cnx = mysql.connector.connect(**config)

In [120]:
cursor = cnx.cursor()

In [121]:
cursor.execute("CREATE DATABASE IF NOT EXISTS amelio")

In [122]:
cursor.execute("USE amelio")

In [123]:
cursor.execute("DROP TABLE IF EXISTS hr_policy_type")
cursor.execute("DROP TABLE IF EXISTS hr_policy")

In [124]:
create_hr_policy_table = """
    CREATE TABLE IF NOT EXISTS hr_policy (
        id INT AUTO_INCREMENT PRIMARY KEY,
        policy_name VARCHAR(255) NOT NULL
    )
    """

In [125]:
cursor.execute(create_hr_policy_table)
print("Table 'hr_policy' created successfully or already exists.")

Table 'hr_policy' created successfully or already exists.


In [126]:
insert_hr_policy = "INSERT INTO hr_policy (policy_name) VALUES ('remote')"
cursor.execute(insert_hr_policy)
print("Value 'remote work' inserted into 'hr_policy' table.")

Value 'remote work' inserted into 'hr_policy' table.


In [127]:
hr_policy_id = cursor.lastrowid

In [128]:
create_hr_policy_type_table = """
    CREATE TABLE IF NOT EXISTS hr_policy_type (
        id INT AUTO_INCREMENT PRIMARY KEY,
        policy_type VARCHAR(255) NOT NULL,
        hr_policy_id INT,
        FOREIGN KEY (hr_policy_id) REFERENCES hr_policy(id)
    )
    """
cursor.execute(create_hr_policy_type_table)
print("Table 'hr_policy_type' created successfully or already exists.")

Table 'hr_policy_type' created successfully or already exists.


In [129]:
insert_hr_policy_type = """
    INSERT INTO hr_policy_type (policy_type, hr_policy_id)
    VALUES 
        ('Flexible', %s)
    """
cursor.execute(insert_hr_policy_type, (hr_policy_id,))
print("Values inserted into 'hr_policy_type' table.")


Values inserted into 'hr_policy_type' table.


In [130]:
# Commit the transactions
cnx.commit()

In [131]:
insert_hr_policy_type = """
    INSERT INTO hr_policy_type (policy_type, hr_policy_id)
    VALUES 
        ('non-flexible', %s)
    """
cursor.execute(insert_hr_policy_type, (hr_policy_id,))
cnx.commit()
insert_hr_policy_type = """
    INSERT INTO hr_policy_type (policy_type, hr_policy_id)
    VALUES 
        ('onsite', %s)
    """
cursor.execute(insert_hr_policy_type, (hr_policy_id,))
cnx.commit()
insert_hr_policy_type = """
    INSERT INTO hr_policy_type (policy_type, hr_policy_id)
    VALUES 
        ('strict', %s)
    """
cursor.execute(insert_hr_policy_type, (hr_policy_id,))
cnx.commit()

In [132]:
cursor.execute("SELECT * FROM hr_policy")
hr_policy_data = cursor.fetchall()
print("\nData from 'hr_policy' table:")
for row in hr_policy_data:
    print(row)

# Fetch data from 'hr_policy_type' table
cursor.execute("SELECT * FROM hr_policy_type")
hr_policy_type_data = cursor.fetchall()
print("\nData from 'hr_policy_type' table:")
for row in hr_policy_type_data:
    print(row)


Data from 'hr_policy' table:
(1, 'remote')

Data from 'hr_policy_type' table:
(1, 'Flexible', 1)
(2, 'non-flexible', 1)
(3, 'onsite', 1)
(4, 'strict', 1)


In [133]:
cursor.close()
cnx.close()

In [25]:
!pip install fpdf

  Using cached fpdf-1.7.2.tar.gz (39 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=a624d53c0b4d393653b0280b0ffb737ff357ed3523815e87b15ec137ad3ec4db
  Stored in directory: /Users/sunilthapa/Library/Caches/pip/wheels/44/35/8b/86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf


In [63]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "Flexible Work Policy", 0, 1, "C")

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, 0, 1, "L")

    def chapter_body(self, body):
        self.set_font("Arial", size=12)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_checkbox(self, x, y, text, width=185):
        self.set_xy(x, y)
        self.set_font("Arial", size=12)
        self.cell(10, 10, "-", 0, 0)
        self.set_x(x + 10)
        self.multi_cell(width, 10, text, 0, 1)

In [64]:
# Create instance of PDF class
pdf = PDF()

In [65]:
# Add a page
pdf.add_page()

In [66]:
# Add policy text
policy_text = """
Flexible Work Policy

1. Objective: The objective of this policy is to outline the provisions for flexible work arrangements, including remote work, flexible hours, and hybrid work setups.

2. Eligibility: All full-time and part-time employees who have completed their probationary period are eligible to request flexible work arrangements.

3. Procedure:
   - Employees should submit a written request to their immediate supervisor.
   - The request must include details of the proposed arrangement and the reasons for the request.
   - Supervisors will review the request and may consult with HR before making a decision.

4. Approval: Approval of flexible work arrangements is at the discretion of management and will be based on the role, department needs, and individual performance.

5. Review and Monitoring:
   - Flexible work arrangements will be reviewed periodically to ensure they continue to meet the needs of the organization and the employee.
   - Employees may be required to attend the office on specific days or for specific meetings as determined by their supervisor.

6. Termination: Management reserves the right to terminate or modify flexible work arrangements with reasonable notice.

7. Responsibilities:
   - Employees must ensure they have a suitable work environment and necessary equipment.
   - Employees are responsible for maintaining productivity and meeting performance standards.

8. Confidentiality: Employees must adhere to the organization's confidentiality and data protection policies while working flexibly.

9. Health and Safety: Employees must comply with health and safety guidelines while working from home or any other location.

10. Communication: Employees must remain accessible and maintain regular communication with their team and supervisor.

This policy is subject to change based on organizational needs and will be reviewed annually.
"""

In [67]:
pdf.chapter_body(policy_text)

# Add checkboxes
pdf.add_checkbox(10, pdf.get_y(), "Flexible hours: Employees can choose their start and end times, with a mandatory core period (e.g., from 10 AM to 3 PM).")
pdf.add_checkbox(10, pdf.get_y(), "Remote work: e.g. Employees can work remotely up to 2 days per week, with their manager's approval.")
pdf.add_checkbox(10, pdf.get_y(), "Compressed workweek: Employees can work their weekly hours in fewer days (e.g., four days instead of five).")
pdf.add_checkbox(10, pdf.get_y(), "Part-time work: Employees can choose to work part-time for a determined or undetermined period, with their manager's approval.")
pdf.add_checkbox(10, pdf.get_y(), "Unpaid leave: Employees can take unpaid leave in addition to their paid leave, subject to approval.")
pdf.add_checkbox(10, pdf.get_y(), "Irregular schedule: Employees can follow a personalized schedule where their working hours are not fixed from day to day, but their total working hours and pay remain the same.")
pdf.add_checkbox(10, pdf.get_y(), "Job sharing: Two employees can share the responsibilities of a single full-time position, with reduced working hours and shared responsibilities.")

# Save the pdf with name .pdf
pdf.output("Flexible_Work_Policy.pdf")

print("PDF created and saved as Flexible_Work_Policy.pdf")


PDF created and saved as Flexible_Work_Policy.pdf
